# Sketchpad - Omgevingsverordening

This notebook was used to wrangle with the Province of Utrecht's omgevingsverordening and its embedding into polygons (.gml files). A reference file was made to more quickly be able to find which ID's were housed in which file of the many (sub)-directories the data houses. It'd be better to make a proper database out of it, if given the time to execute.

TO-DO: REFERENCE WHERE IT STANDS AND FILENAME OF DATASET

As this consists largely of code not used in the final pipeline, it lacks proper documentation, and is largely kept for archival purposes.

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from lxml import etree

## Getting through the dataset

There's an implicit hierarchy in the data:
- Regel --> Regeltekst --> Tekst
- Regel --> (Gebiedsaanwijzing) --> (Gebiedengroep) --> Gebied --> GML


Which is bidirectional, so also:
- Tekst --> Regeltekst --> Regel
- GML --> Gebied --> (Gebiedengroep) --> (Gebiedsaanwijzing) --> Regel

Meaning, if you have a piece of GML with coordinates, wanting to know what rules and laws it touches, you have to take a path like this: GML --> Gebied --> (Gebiedengroep) --> (Gebiedsaanwijzing) --> Regel --> Regeltekst --> Tekst



The location of documents for each piece is as follows:
- Regel: `/OW-bestanden/instructieregels.xml`, `OW-bestanden/regelsvooriedereen.xml`
- Regeltekst: `/OW-bestanden/regelteksten.xml`
- Tekst: `/Regeling/Tekst.xml`
- Gebiedsaanwijzing: `/OW-bestanden/gebiedsaanwijzingen.xml`
- Gebiedengroep: `/OW-bestanden/gebiedengroepen.xml`
- Gebied: `/OW-bestanden/gebieden.xml`
- GML: found in various `IO-XXXXXX/` folders, each likely corresponding to a different theme but that needs to be verified

## To make this dataset managable

- Set up a PostgreSQL database
    - Containerise it within the research cloud workspace
    - Then start piecing it together to put it in database (first just big batch one table)
    - Then start cutting up the table into smaller tables

In [ ]:
# Data base path
data_path = '../data/spatial_genai_storage/data_omgevingsverordening/data_fase_2/omgevingsverordening/'

# Load in the data
instructieregels = etree.parse(f'{data_path}OW-bestanden/instructieregels.xml')
iedereenregels = etree.parse(f'{data_path}OW-bestanden/regelsvooriedereen.xml')
regelteksten = etree.parse(f'{data_path}OW-bestanden/regelteksten.xml')
teksten = etree.parse(f'{data_path}Regeling/Tekst.xml')
gebiedengroepen = etree.parse(f'{data_path}OW-bestanden/gebiedengroepen.xml')
gebieden = etree.parse(f'{data_path}OW-bestanden/gebieden.xml')

In [4]:
# Get the root element
root = instructieregels.getroot()

# Print the tag of the root element
print(f"Root tag: {root.tag}")

# Get all child elements of the root
children = list(root)
print(f"Number of children: {len(children)}")

# Find all elements with a specific tag using XPath
regel_elements = instructieregels.xpath("//Regel")
print(f"Number of Regel elements: {len(regel_elements)}")

# Get element attributes
if regel_elements:
    print(f"First Regel attributes: {regel_elements[0].attrib}")

# Find elements by attribute value
elements_with_id = instructieregels.xpath("//*[@id]")
print(f"Number of elements with id attribute: {len(elements_with_id)}")

# Get text content of an element
if regel_elements and len(regel_elements[0]) > 0:
    print(f"First child text of first Regel: {regel_elements[0][0].text}")

# Convert element to string
if regel_elements:
    regel_as_string = etree.tostring(regel_elements[0], pretty_print=True).decode()
    print(f"First Regel as string (first 100 chars): {regel_as_string[:100]}")

# Count elements by namespace
namespaces = {prefix: uri for prefix, uri in root.nsmap.items() if prefix is not None}
print(f"Namespaces used: {namespaces}")

Root tag: {http://www.geostandaarden.nl/imow/bestanden/deelbestand}owBestand
Number of children: 1
Number of Regel elements: 0
Number of elements with id attribute: 0
Namespaces used: {'rol': 'http://www.geostandaarden.nl/imow/regelsoplocatie', 'schemaLocation': 'http://www.geostandaarden.nl/imow/bestanden/deelbestand', 'rg': 'http://www.geostandaarden.nl/imow/regelingsgebied', 'da': 'http://www.geostandaarden.nl/imow/datatypenalgemeen', 'l': 'http://www.geostandaarden.nl/imow/locatie', 'sym': 'http://www.geostandaarden.nl/imow/symbolisatie', 'xsi': 'http://www.w3.org/2001/XMLSchema-instance', 'kaart': 'http://www.geostandaarden.nl/imow/kaart', 'xlink': 'http://www.w3.org/1999/xlink', 'ow-dc': 'http://www.geostandaarden.nl/imow/bestanden/deelbestand', 'ga': 'http://www.geostandaarden.nl/imow/gebiedsaanwijzing', 'sl': 'http://www.geostandaarden.nl/bestanden-ow/standlevering-generiek', 'p': 'http://www.geostandaarden.nl/imow/pons', 'regels': 'http://www.geostandaarden.nl/imow/regels', 'o

In [5]:
namespaces = {
    'ow-dc': 'http://www.geostandaarden.nl/imow/bestanden/deelbestand',
    'sl': 'http://www.geostandaarden.nl/bestanden-ow/standlevering-generiek',
    'regels': 'http://www.geostandaarden.nl/imow/regels',
    'rol': 'http://www.geostandaarden.nl/imow/regelsoplocatie',
    'rg': 'http://www.geostandaarden.nl/imow/regelingsgebied',
    'l': 'http://www.geostandaarden.nl/imow/locatie',
    'ga': 'http://www.geostandaarden.nl/imow/gebiedsaanwijzing',
    'k': 'http://www.geostandaarden.nl/imow/kaart',
    'kaart': 'http://www.geostandaarden.nl/imow/kaart',
    'p': 'http://www.geostandaarden.nl/imow/pons',
    'vt': 'http://www.geostandaarden.nl/imow/vrijetekst',
    's': 'http://www.geostandaarden.nl/imow/symbolisatie',
    'sym': 'http://www.geostandaarden.nl/imow/symbolisatie',
    'da': 'http://www.geostandaarden.nl/imow/datatypenalgemeen',
    'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
    'xlink': 'http://www.w3.org/1999/xlink',
    'ow': 'http://www.geostandaarden.nl/imow/owobject',
    'op': 'http://www.geostandaarden.nl/imow/opobject',
    'xs': 'http://www.w3.org/2001/XMLSchema',
    'basisgeo' : 'http://www.geostandaarden.nl/basisgeometrie/1.0',
    'gml' : 'http://www.opengis.net/gml/3.2',
    'geo' : 'https://standaarden.overheid.nl/stop/imop/geo/'
}

In [6]:
instructieregels_list = [regel for regel in root.findall('.//regels:Instructieregel', namespaces=namespaces)]
instructieregels_locaties = [locatie_ref for instructieregel in instructieregels_list for locatie_ref in instructieregel.findall('.//l:LocatieRef', namespaces=namespaces)]

print(f"Number of instructieregels: {len(instructieregels_list)}")
print(f"Number of LocatieRef: {len(instructieregels_locaties)}")

# Make a dict of instructieregels_list with each having a list of instructieregels_locaties if it has any
instructieregels_dict = {}
    

Number of instructieregels: 250
Number of LocatieRef: 313


In [7]:
# get the text of instructieregel id
instructieregel_id = instructieregels_list[0].findall('regels:identificatie', namespaces=namespaces)[0].text

# get the text of lid artikel
instructieregel_artikellid = instructieregels_list[0].findall('regels:artikelOfLid/regels:RegeltekstRef', namespaces=namespaces)[0].get('{http://www.w3.org/1999/xlink}href')

# get the locatieaanduiding (can be multiple!)
instructieregel_locatie = [locatie.get('{http://www.w3.org/1999/xlink}href') for locatie in instructieregels_list[30].findall('regels:locatieaanduiding/l:LocatieRef', namespaces=namespaces)]

# get the gebiedsaanwijzing (can be multiple!)
instructieregel_gebiedsaanwijzing = [aanwijzing.get('{http://www.w3.org/1999/xlink}href') for aanwijzing in instructieregels_list[30].findall('regels:gebiedsaanwijzing/ga:GebiedsaanwijzingRef', namespaces=namespaces)]


print(f"ID: {instructieregel_id}")
print(f"Artikel lid: {instructieregel_artikellid}")
print(f"Locatieaanduiding: {instructieregel_locatie}")
print(f"Gebiedsaanwijzing: {instructieregel_gebiedsaanwijzing}")

ID: nl.imow-pv26.juridischeregel.003000000000000000000146
Artikel lid: nl.imow-pv26.regeltekst.003000000000000000000146
Locatieaanduiding: ['nl.imow-pv26.gebiedengroep.003000000000000000000561', 'nl.imow-pv26.gebiedengroep.003000000000000000000624', 'nl.imow-pv26.gebiedengroep.003000000000000000000677']
Gebiedsaanwijzing: ['nl.imow-pv26.gebiedsaanwijzing.003000000000000000000155', 'nl.imow-pv26.gebiedsaanwijzing.003000000000000000000201', 'nl.imow-pv26.gebiedsaanwijzing.003000000000000000000242']


In [8]:
target_value = instructieregel_locatie[0]
xpath_query = f".//l:Gebiedengroep[l:identificatie = '{target_value}']"

gebiedengroep = gebiedengroepen.xpath(xpath_query, namespaces=namespaces)
gebiedengroep_id = gebiedengroep[0].find("l:identificatie", namespaces=namespaces).text
gebiedengroep_noemer = gebiedengroep[0].find("l:noemer", namespaces=namespaces).text
gebiedengroep_gebieden = gebiedengroep[0].findall('l:groepselement/l:GebiedRef', namespaces=namespaces)
gebiedengroep_gebiedreferenties = [gebied.get('{http://www.w3.org/1999/xlink}href') for gebied in gebiedengroep[0].findall('l:groepselement/l:GebiedRef', namespaces=namespaces)]

print(f"Nummer van gebiedengroepen met identificatie '{target_value}': {len(gebiedengroep)}")
print(f"Noemer van gebiedengroep: '{gebiedengroep_noemer}'")
print(f'Identificatie = {gebiedengroep_id}')
print(f"Nummer van gebieden in gebiedengroep '{target_value}': {len(gebiedengroep_gebieden)}")
print(f"Gebiedengroep gebiedreferenties: {gebiedengroep_gebiedreferenties}")

Nummer van gebiedengroepen met identificatie 'nl.imow-pv26.gebiedengroep.003000000000000000000561': 1
Noemer van gebiedengroep: 'Historische buitenplaatszone'
Identificatie = nl.imow-pv26.gebiedengroep.003000000000000000000561
Nummer van gebieden in gebiedengroep 'nl.imow-pv26.gebiedengroep.003000000000000000000561': 12
Gebiedengroep gebiedreferenties: ['nl.imow-pv26.gebied.003000000000000000000561e77855', 'nl.imow-pv26.gebied.003000000000000000000561e77861', 'nl.imow-pv26.gebied.003000000000000000000561e77864', 'nl.imow-pv26.gebied.003000000000000000000561e77857', 'nl.imow-pv26.gebied.003000000000000000000561e77866', 'nl.imow-pv26.gebied.003000000000000000000561e77859', 'nl.imow-pv26.gebied.003000000000000000000561e77858', 'nl.imow-pv26.gebied.003000000000000000000561e77863', 'nl.imow-pv26.gebied.003000000000000000000561e77856', 'nl.imow-pv26.gebied.003000000000000000000561e77860', 'nl.imow-pv26.gebied.003000000000000000000561e77862', 'nl.imow-pv26.gebied.003000000000000000000561e7786

In [9]:
target_value = gebiedengroep_gebiedreferenties[0]
xpath_query = f".//l:Gebied[l:identificatie = '{target_value}']"

gebied = gebieden.xpath(xpath_query, namespaces=namespaces)
gebied_geometriereferentie = gebied[0].find("l:geometrie/l:GeometrieRef", namespaces=namespaces).get('{http://www.w3.org/1999/xlink}href')

print(f"Geometriereferentie van gebiedreferentie '{target_value}': {gebied_geometriereferentie}")

Geometriereferentie van gebiedreferentie 'nl.imow-pv26.gebied.003000000000000000000561e77855': 3d93207c-8f4a-46c8-bf9b-19310ca35183


In [10]:
import os

# read in the gml file
example_path = f'{data_path}IO-18359756555207046090/locaties_Landbouwstabiliseringsgebied-v1.gml'
example_gml = etree.parse(example_path)

filename = os.path.basename(example_path)
print(f"Filename: {filename}")
theme = filename[len('locaties_'):-4]
print(f"Theme: {theme}")
elements_with_id = example_gml.findall(".//*basisgeo:id", namespaces=namespaces)
print(f"Number of elements with id attribute: {len(elements_with_id)}")
print(f"{elements_with_id[0].text}")

Filename: locaties_Landbouwstabiliseringsgebied-v1.gml
Theme: Landbouwstabiliseringsgebied-v1
Number of elements with id attribute: 812
df9f4060-37e3-49af-a621-37a412bfea28


### Make reference file

Making a reference file to speed up the look-up of ID's in files within the many nested directories. Preferential to make a proper database, but given time a quick reference doc will be good enough for the prototype.

In [ ]:
import os
import glob
from lxml import etree
import json
from pathlib import Path
import multiprocessing

def process_file(file_path, namespaces):
    """Process a single GML file and extract reference IDs"""
    filename = os.path.basename(file_path)
    
    if filename.startswith('locaties_') and filename.endswith('.gml'):
        theme = filename[len('locaties_'):-4]
    else:
        theme = os.path.basename(os.path.dirname(file_path))
    
    local_refs = {}
    
    try:
        tree = etree.parse(file_path)
        
        # Find all elements with an ID attribute
        elements_with_id = tree.findall(".//*basisgeo:id", namespaces=namespaces)
        
        for element in elements_with_id:
            ref_id = element.text
            local_refs[ref_id] = {
                'file_path': file_path,
                'theme': theme
            }
            
        return local_refs
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return {}

def build_reference_index(data_path, namespaces):
    """Build an index that maps reference IDs to file locations"""
    reference_index = {}
    
    gml_files = glob.glob(f"{data_path}/**/**.gml", recursive=True)
    print(f"Found {len(gml_files)} GML files to process")
    
    # Create a partial function with the namespaces argument already set
    from functools import partial
    process_with_namespaces = partial(process_file, namespaces=namespaces)
    
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.map(process_with_namespaces, gml_files)
    
    for result in results:
        reference_index.update(result)
        
    print(f"Indexed {len(reference_index)} unique reference IDs")
    
    with open(f"{data_path}/reference_index.json", 'w') as f:
        json.dump(reference_index, f)
        
    return reference_index

def find_reference(reference_id, index):
    """Look up a reference ID in the index"""
    if reference_id in index:
        return index[reference_id]
    return None

In [12]:
index = build_reference_index(data_path, namespaces)

Found 122 GML files to process
Indexed 15912 unique reference IDs


In [13]:
find_reference(gebied_geometriereferentie, index)

{'file_path': 'data/spatial_genai_4_spatial_fase2/data_fase_2/akn_nl_act_pv26_2022_omgevingsverordening_download DSO/IO-16764149682679986215/locaties_Historischebuitenplaatszone-v1.gml',
 'theme': 'Historischebuitenplaatszone-v1'}

In [ ]:
# From a location, get all relevant information
location_id = '3d93207c-8f4a-46c8-bf9b-19310ca35183'

location_reference = find_reference(location_id, index)
print(f"File path: {location_reference['file_path']}") 
print(f"Theme: {location_reference['theme']}")

def find_gebied()


File path: data/spatial_genai_4_spatial_fase2/data_fase_2/akn_nl_act_pv26_2022_omgevingsverordening_download DSO/IO-16764149682679986215/locaties_Historischebuitenplaatszone-v1.gml
Theme: Historischebuitenplaatszone-v1


geoJSON export <-- vanuit hun acrgis omgeving
database in beheer door derden
NGR heeft veelal GML? ipv GeoJSON? kan een verschil in zitten mbt publieke API uittrekken en wat ze via arcGIS kunnen krijgen.
metadata staat in esri niet direct gekoppeld aan de datasets?